### 读取源数据

In [58]:
def read_file(file_name):
    """读取对应数据文件的数据"""
    fp = open(file_name)
    # 列标题
    columns = fp.readline().split("\t")[1:]
    row_names, data = [], []
    for line in fp:
        row_name, *row_data = line.split("\t")
        row_names.append(row_name)
        data.append(list(map(float, row_data)))
    fp.close()
    return row_names, columns, data

In [27]:
file_name = "./data/blogdata.txt"

### 定义紧密度（closeness）
使用皮尔逊相关度

In [30]:
from math import sqrt, pow

def pearson(v1, v2):
    # 简单求和
    sum1, sum2 = sum(v1), sum(v2)
    
    # 计算平方和
    sum1_pow, sum2_pow = sum((pow(num, 2) for num in v1)), sum((pow(num, 2) for num in v2))
    
    # 求乘积之和
    p_sum = sum((num1 * num2 for num1, num2 in zip(v1, v2)))
    
    # 计算 r 
    n = len(v1)
    num  = p_sum - (sum1 * sum2 / n)
    den = sqrt((sum1_pow - pow(sum1, 2) / n) * (sum2_pow - pow(sum2, 2) / n))
    if den == 0:
        return 0
    return  1.0 - num / den

### 定义树状数据

In [69]:
class Bicluster:
    def __init__(self, vec, left=None, right=None, distance=0.0, id=None):
        self.vec = vec
        self.left = left
        self.right = right
        self.id = id
        self.distance = distance
    
    @property
    def is_leaf_node(self):
        return self.left is None and self.right is None

### 分级聚类主函数

In [59]:
from itertools import combinations

def hcluster(rows, distance=pearson):
    distances = {}
    current_clust_id = -1
    # 生成树类数据
    clust = [Bicluster(row, id=i) for i, row in enumerate(rows)]
    
    while len(clust) > 1:
        closest = 2
        pair = (-1, -1)
        for i1, i2 in combinations(range(len(clust)), 2):
            id_pair = (clust[i1].id, clust[i2].id)
            if id_pair not in distances:
                distances[id_pair] = distance(clust[i1].vec, clust[i2].vec)
                
            d = distances[id_pair]
            if d < closest:
                closest, pair = d, (i1, i2)
                
        new_mean_bs = Bicluster(
            vec=[sum(vec)/2 for vec in zip(clust[pair[0]].vec, clust[pair[1]].vec)],
            left=clust[pair[0]],
            right=clust[pair[1]],
            id=current_clust_id,
            distance=closest
        )
        
        current_clust_id -= 1
        del clust[pair[1]]
        del clust[pair[0]]
        
        clust.append(new_mean_bs)    
    
    return clust[0]

In [60]:
blog_names, words, data = read_file(file_name=file_name)

In [70]:
res = hcluster(data)

In [56]:
res.id

-54

In [49]:
### 分级展示

In [54]:
def print_clust(clust: Bicluster, labels=None, n=0):
    [print(" ", end="") for i in range(n)]
    if clust.id < 0:
        # 分支
        print('-')
    else:
        # 叶节点
        if labels is None:
            print(clust.id)
        else:
            print(labels[clust.id])
    
    # 打印左分支和右分支
    if clust.left is not None:
        print_clust(clust.left, labels, n=n+1)
    if clust.right is not None:
        print_clust(clust.right, labels, n=n+1)
        
    

In [ ]:
print_clust(res, labels=blog_names)

In [62]:
from PIL import Image, ImageDraw

In [71]:
def get_height(clust: Bicluster):
    # 叶节点高度为 1
    if clust.is_leaf_node:
        return 1
    
    return get_height(clust.left) + get_height(clust.right)

def get_depth(clust: Bicluster):
    # 叶节点的距离为 0
    if clust.is_leaf_node:
        return 0
    return max(get_depth(clust.left), get_depth(clust.right)) + clust.distance

In [73]:
def draw_dendrogram(clust, labels, jpeg='clusters.jpg'):
    # 高宽
    h = get_height(clust) * 20
    w = 1200
    depth = get_depth(clust)
    scaling = float(w - 150) / depth
    
    img = Image.new('RGB', (w, h), (255, 255, 255))
    draw = ImageDraw.Draw(img)
    
    draw.line((0, h/2, 10, h/2), fill=(255, 0, 0))
    
    draw_node(draw, clust, 10, (h/2), scaling, labels)
    img.save(jpeg, 'JPEG')

def draw_node(draw, clust, x, y, scaling, labels):
    if clust.id < 0:
        h1 = get_height(clust.left) * 20
        h2 = get_height(clust.right) * 20
        top = y - (h1 + h2) / 2
        bottom = y + (h1 + h2)  / 2
        
        ll = clust.distance * scaling
        
        # Vertical line from this cluster to children
        draw.line((x, top + h1 / 2, x, bottom - h2 / 2), fill=(255, 0, 0))

        # Horizontal line to left item
        draw.line((x, top + h1 / 2, x + ll, top + h1 / 2), fill=(255, 0, 0))

        # Horizontal line to right item
        draw.line((x, bottom - h2 / 2, x + ll, bottom - h2 / 2), fill=(255, 0, 0))
        
        # Call the function to draw the left and right nodes
        draw_node(draw, clust.left, x + ll, top + h1 / 2, scaling, labels)
        draw_node(draw, clust.right, x + ll, bottom - h2 / 2, scaling, labels)
    else:
        # If this is an endpoint, draw the item label
        draw.text((x + 5, y - 7), labels[clust.id], (0, 0, 0))

In [74]:
draw_dendrogram(clust=res, labels=blog_names, jpeg='blog_clust.jpg')

In [75]:
def rotate_matrix(data):
    new_data = []
    for i in range(len(data[0])):
        new_row = [data[j][i] for j in range(len(data))]
        new_data.append(new_row)
    return new_data
    

In [95]:
from random import random

In [76]:
data2 = rotate_matrix(data)

In [122]:
def kcluster(rows, distance=pearson, k=4, max_iter=100):
    # 确定点的最大值和最小值
    ranges = []
    for i in range(len(rows[0])):
        tmp = [row[i] for row in data]
        ranges.append((min(tmp), max(tmp)))

    clusters = [[random() * (max_v - min_v) + min_v for min_v, max_v in ranges] for j in range(k)]  # 随机创建 k 个点     

    last_matches = None
    data_len = len(rows[0])
    for t in range(max_iter):
        print(f"迭代第 {t + 1} 次")
        # 寻找离最中心接近的点
        best_matches = [[] for i in range(k)]
        for j, row in enumerate(rows):
            cluster_index, _ = min([(i, distance(cluster, row)) for i, cluster in enumerate(clusters)], key=lambda x: x[1])
            best_matches[cluster_index].append(j)
        # 如果没变化了，就停止
        if best_matches == last_matches:
            print("达到最佳效果")
            break
        last_matches = best_matches
        # 中心点移到成员的平均处
        for match in best_matches:
            avg = []
            for i in range(data_len):
                avg.append(sum(rows[index][i] for index in match) / data_len)
            clusters[match_index] = avg
    return best_matches

In [123]:
r = kcluster(data, k=4)
print(r)

迭代第 1 次
迭代第 2 次
迭代第 3 次
迭代第 4 次
达到最佳效果
[[18, 54], [5, 7, 9, 10, 11, 15, 19, 24, 26, 27, 37, 38, 40, 42, 44, 47, 48, 53], [6, 13], [0, 1, 2, 3, 4, 8, 12, 14, 16, 17, 20, 21, 22, 23, 25, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 41, 43, 45, 46, 49, 50, 51, 52]]


### 定义 Tanimoto 系数

In [126]:
def tanimoto(v1, v2):
    c1, c2, shr = 0, 0, 0
    for d1, d2 in zip(v1, v2):
        c1 += d1
        c2 += d2
        shr += (d1 and d2)
    return 1.0 - (float(shr) / (c1+c2-shr))


### 对结果聚类

In [127]:
wants, people, data = read_file('data/zebo.txt')
clust = hcluster(data, distance=tanimoto)
draw_dendrogram(clust, wants)